# Waymo Open Dataset Motion Tutorial

- Website: https://waymo.com/open
- GitHub: https://github.com/waymo-research/waymo-open-dataset

This tutorial demonstrates:
- How to decode and interpret the data.
- How to train a simple model with Tensorflow.

Visit the [Waymo Open Dataset Website](https://waymo.com/open) to download the full dataset.

To use, open this notebook in [Colab](https://colab.research.google.com).

Uncheck the box "Reset all runtimes before running" if you run this colab directly from the remote kernel. Alternatively, you can make a copy before trying to run it by following "File > Save copy in Drive ...".

# Package installation

In [1]:
!pip install waymo-open-dataset-tf-2-12-0==1.6.4

Looking in indexes: https://mirrors.aliyun.com/pypi/simple


# Imports and global definitions

## 数据文件的路径

In [3]:
# Data location. Please edit.

# A tfrecord containing tf.Example protos as downloaded from the Waymo dataset
# webpage.

# Replace this path with your own tfrecords.
FILENAME = '/home/heihuhu/PycharmProjects/waymo/data/uncompressed_tf_example_training_training_tfexample.tfrecord-00000-of-01000'

## 定义features_description用来读取tf文件

In [4]:
import math
import os
import uuid
import time

from matplotlib import cm
import matplotlib.animation as animation
import matplotlib.pyplot as plt

import numpy as np
from IPython.display import HTML
import itertools
import tensorflow as tf

from google.protobuf import text_format
from waymo_open_dataset.metrics.ops import py_metrics_ops
from waymo_open_dataset.metrics.python import config_util_py as config_util
from waymo_open_dataset.protos import motion_metrics_pb2

# If you use a custom conversion from Scenario to tf.Example, set the correct
# number of map samples here.
num_map_samples = 30000

# Example field definition
roadgraph_features = {
    'roadgraph_samples/dir': tf.io.FixedLenFeature(
        [num_map_samples, 3], tf.float32, default_value=None
    ),
    'roadgraph_samples/id': tf.io.FixedLenFeature(
        [num_map_samples, 1], tf.int64, default_value=None
    ),
    'roadgraph_samples/type': tf.io.FixedLenFeature(
        [num_map_samples, 1], tf.int64, default_value=None
    ),
    'roadgraph_samples/valid': tf.io.FixedLenFeature(
        [num_map_samples, 1], tf.int64, default_value=None
    ),
    'roadgraph_samples/xyz': tf.io.FixedLenFeature(
        [num_map_samples, 3], tf.float32, default_value=None
    ),
}
# Features of other agents.
state_features = {
    'state/id':
        tf.io.FixedLenFeature([128], tf.float32, default_value=None),
    'state/type':
        tf.io.FixedLenFeature([128], tf.float32, default_value=None),
    'state/is_sdc':
        tf.io.FixedLenFeature([128], tf.int64, default_value=None),
    'state/tracks_to_predict':  #一个标志向量，用于指示要预测哪些对象对应于 state/XX 张量的第一个维度。最多可选择 8 个对象。对于训练集和验证集，这些指示提交 proto prediction 中必须存在哪个对象。如果在提交 proto 中没有准确预测这些对象，则提交将失败。此字段也存在于训练集中，作为要训练的对象的参考，但您可以自由选择要训练的其他对象。
        tf.io.FixedLenFeature([128], tf.int64, default_value=None),
    'state/objects_of_interest':
        tf.io.FixedLenFeature([128], tf.int64, default_value=None),

    'state/current/bbox_yaw':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/height':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/length':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/timestamp_micros':  # 每个时间步长的时间戳（以微秒为单位）。
        tf.io.FixedLenFeature([128, 1], tf.int64, default_value=None),
    'state/current/valid':  #功能 state/current/XX 的所有元素的有效标志。如果设置为 1，则元素将填充有效数据，否则将填充 -1。{0, 1}.指示 state/current/XX 张量的每个元素的数据是否填充了有效数据（而不是用 -1 填充）。
        tf.io.FixedLenFeature([128, 1], tf.int64, default_value=None),
    'state/current/vel_yaw':  #每个对象的速度向量在每个时间步长的偏转角。角度（以弧度为单位）。这是在 X-Y 平面中沿 X 方向旋转单位向量以与对象边界框的速度向量对齐的角度。
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/velocity_x':  #每个时间步长处对象速度的 x 分量。
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/velocity_y':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/width':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/x':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/y':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/z':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/speed':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),

    'state/future/bbox_yaw':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/height':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/length':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/timestamp_micros':
        tf.io.FixedLenFeature([128, 80], tf.int64, default_value=None),
    'state/future/valid':
        tf.io.FixedLenFeature([128, 80], tf.int64, default_value=None),
    'state/future/vel_yaw':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/velocity_x':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/velocity_y':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/width':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/x':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/y':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/z':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/speed':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),

    'state/past/bbox_yaw':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/height':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/length':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/timestamp_micros':
        tf.io.FixedLenFeature([128, 10], tf.int64, default_value=None),
    'state/past/valid':
        tf.io.FixedLenFeature([128, 10], tf.int64, default_value=None),
    'state/past/vel_yaw':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/velocity_x':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/velocity_y':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/width':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/x':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/y':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/z':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/speed':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
}

traffic_light_features = {
    'traffic_light_state/current/state':
        tf.io.FixedLenFeature([1, 16], tf.int64, default_value=None),
    'traffic_light_state/current/valid':
        tf.io.FixedLenFeature([1, 16], tf.int64, default_value=None),
    'traffic_light_state/current/id':
        tf.io.FixedLenFeature([1, 16], tf.int64, default_value=None),
    'traffic_light_state/current/timestamp_micros':
        tf.io.FixedLenFeature([1], tf.int64, default_value=None),
    'traffic_light_state/current/x':
        tf.io.FixedLenFeature([1, 16], tf.float32, default_value=None),
    'traffic_light_state/current/y':
        tf.io.FixedLenFeature([1, 16], tf.float32, default_value=None),
    'traffic_light_state/current/z':
        tf.io.FixedLenFeature([1, 16], tf.float32, default_value=None),

    'traffic_light_state/future/state':
        tf.io.FixedLenFeature([80, 16], tf.int64, default_value=None),
    'traffic_light_state/future/valid':
        tf.io.FixedLenFeature([80, 16], tf.int64, default_value=None),
    'traffic_light_state/future/id':
        tf.io.FixedLenFeature([80, 16], tf.int64, default_value=None),
    'traffic_light_state/future/timestamp_micros':
        tf.io.FixedLenFeature([80], tf.int64, default_value=None),
    'traffic_light_state/future/x':
        tf.io.FixedLenFeature([80, 16], tf.float32, default_value=None),
    'traffic_light_state/future/y':
        tf.io.FixedLenFeature([80, 16], tf.float32, default_value=None),
    'traffic_light_state/future/z':
        tf.io.FixedLenFeature([80, 16], tf.float32, default_value=None),

    'traffic_light_state/past/state':
        tf.io.FixedLenFeature([10, 16], tf.int64, default_value=None),
    'traffic_light_state/past/valid':
        tf.io.FixedLenFeature([10, 16], tf.int64, default_value=None),
    'traffic_light_state/past/id':
        tf.io.FixedLenFeature([10, 16], tf.int64, default_value=None),
    'traffic_light_state/past/timestamp_micros':
        tf.io.FixedLenFeature([10], tf.int64, default_value=None),
    'traffic_light_state/past/x':
        tf.io.FixedLenFeature([10, 16], tf.float32, default_value=None),
    'traffic_light_state/past/y':
        tf.io.FixedLenFeature([10, 16], tf.float32, default_value=None),
    'traffic_light_state/past/z':
        tf.io.FixedLenFeature([10, 16], tf.float32, default_value=None),
}

features_description = {}
features_description.update(roadgraph_features)
features_description.update(state_features)
features_description.update(traffic_light_features)

2025-04-07 14:06:31.177037: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-07 14:06:31.199376: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-07 14:06:31.514594: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Visualize TF Example sample

## Create Dataset.

In [5]:
dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
data = next(dataset.as_numpy_iterator())
parsed = tf.io.parse_single_example(data, features_description)

2025-04-07 14:06:35.617699: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-07 14:06:35.640536: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-04-07 14:06:35.666912: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Pla

### 查看数据结构

展示parsed是什么东西。`state/objects_of_interest`表示与主车有交互的两辆车，而不是一辆交互车一辆主车。

In [21]:
# parsed是tf文件转换为的字典
print(f"type of parsed: {type(parsed)}")
print(f"length of parsed: {len(parsed)}")

print(f"traffic_light_state/past/z: {parsed['traffic_light_state/past/z']}")
print(f"shape of traffic_light_state/past/z: {parsed['traffic_light_state/past/z'].shape}")

print(f"state/objects_of_interest: {parsed['state/objects_of_interest']}")
print(f"shape of state/objects_of_interest: {parsed['state/objects_of_interest'].shape}")

type of parsed: <class 'dict'>
length of parsed: 70
traffic_light_state/past/z: [[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]]
shape of traffic_light_state/past/z: (10, 16)
state/objects_of_interest: [1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0

**查看主车id、跟主车有交互的车辆。**

`state/id`: An integer ID for each object.

`state/is_sdc`: A mask to indicate if the object is the autonomous vehicle.

查找主车的ID，然后注意到所有有交互的车辆都是与主车交互的——数据集已经给出每帧与主车交互的actor。

In [7]:
print(f"state/id: {parsed['state/id']}")
print(f"shape of state/id: {parsed['state/id'].shape}")

print(f"state/is_sdc: {parsed['state/is_sdc']}")
print(f"shape of state/is_sdc: {parsed['state/is_sdc'].shape}")

sdc_index = np.where(parsed['state/is_sdc'].numpy() == 1)[0]
print(f"sdc_index: {sdc_index}")
if len(sdc_index) > 0:
    sdc_index = sdc_index[0]  # 取第一个主车（通常只有一个）
else:
    sdc_index = None  # 无主车的情况

print(f"主车ID为：{parsed['state/id'][sdc_index]}")

state/id: [1773. 1778. 1805. 2651. 2656. 2846. 2650. 1763. 2649. 2653. 2660. 1770.
 2658. 2712. 2701. 2648. 2655. 1764. 1797. 1765. 2719. 2671. 1762. 1782.
 1775. 1767. 1771. 1766. 1768. 1810. 1760. 1774. 2696. 1800. 1787. 2727.
 2679. 1772. 2657. 1784. 1815. 1776. 1842. 1777. 1761. 1832. 1791. 2697.
 1841. 2664. 1798. 1781. 1806. 1779. 1783. 1901. 1847. 1799. 1789. 1786.
 1835. 1780. 1788. 1830. 2654. 1820. 1802. 1890. 1837. 1873. 1833. 1828.
 1881. 1862. 2691. 1825. 1803. 1883. 1822. 1816. 1907. 1856. 1904. 1796.
 1792. 1884. 1795. 1910. 1794. 1821. 1903. 2662. 1902. 1812. 1793. 1839.
 1769. 1814. 1823. 1785. 1818. 1790. 1811. 1838. 1857. 1843. 1886. 1817.
 1824. 1804. 1819. 1840. 1870. 1869. 1826. 1829. 1852. 1859. 1849. 1831.
 1845. 1844. 2667. 1846. 1891. 1895. 1867. 1836.]
shape of state/id: (128,)
state/is_sdc: [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0

**展示type，查看不同类型的actor**

`state/type`: An integer type for each object (Vehicle, Pedestrian, or Cyclist)
- Unset=0, Vehicle=1, Pedestrian=2, Cyclist=3, Other=4

In [8]:
print(f"state/type: {parsed['state/type']}")
print(f"shape of state/type: {parsed['state/type'].shape}")

print(f"state/tracks_to_predict: {parsed['state/tracks_to_predict']}")
print(f"shape of state/tracks_to_predict: {parsed['state/tracks_to_predict'].shape}")

state/type: [1. 1. 1. 2. 2. 1. 2. 1. 2. 2. 2. 1. 2. 2. 2. 2. 2. 1. 1. 1. 2. 2. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 2. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2.
 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 2. 1. 1. 1. 1. 1.]
shape of state/type: (128,)
state/tracks_to_predict: [1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
shape of state/tracks_to_predict: (128,)


### 分别获取不同种类actor的下标

配色方案
- Unset=0(gray,spot),
- Vehicle=1(blue,spot),
    - interest(靛蓝色)
    - sdc(red spot)
- Pedestrian=2(yellow,spot),
- Cyclist=3(green,spot),
- Other=4(brown,spot)

In [9]:
def get_actor_indices(decoded_example):
    """
    根据 state/type 数组，返回不同类型的 actor 的下标。
    :param decoded_example: 处理好的数据
    :return:
        {
            'Vehicle': [indices],
            'Pedestrian': [indices],
            'Cyclist': [indices],
            'Other': [indices]
        }
    """
    state_type_array = decoded_example['state/type']
    # 确保输入是 NumPy 数组
    if not isinstance(state_type_array, np.ndarray):
        state_type_array = np.array(state_type_array)

    # 新增：获取主车索引
    sdc_index = np.where(decoded_example['state/is_sdc'].numpy() == 1)[0]
    print(f"sdc_index{sdc_index}")
    if len(sdc_index) > 0:
        sdc_index = sdc_index[0]  # 取第一个主车（通常只有一个）
    else:
        sdc_index = None  # 无主车的情况

    # 新增：获取交互车辆索引
    objects_of_interest = np.where(decoded_example['state/objects_of_interest'].numpy() == 1)[0]
    print(f"objects_of_interest: {objects_of_interest}")

    # 新增：获取to predict车辆索引
    tracks_to_predict = np.where(decoded_example['state/tracks_to_predict'].numpy() == 1)[0]
    print(f"state/tracks_to_predict: {tracks_to_predict}")

    # 使用 NumPy 的向量化操作快速获取下标
    indices = {
        'Vehicle': np.where(state_type_array == 1)[0].tolist(),
        'Pedestrian': np.where(state_type_array == 2)[0].tolist(),
        'Cyclist': np.where(state_type_array == 3)[0].tolist(),
        'Other': np.where(state_type_array == 4)[0].tolist(),
        'sdc_index': sdc_index,
        'objects_of_interest': objects_of_interest,
        'tracks_to_predict': tracks_to_predict
    }

    return indices


index = get_actor_indices(parsed)
print(index)
print(index['objects_of_interest'])
print(index['tracks_to_predict'])

sdc_index[5]
objects_of_interest: [0 4]
state/tracks_to_predict: [0 1 2 3 4]
{'Vehicle': [0, 1, 2, 5, 7, 11, 17, 18, 19, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 37, 39, 40, 41, 42, 43, 44, 45, 46, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 123, 124, 125, 126, 127], 'Pedestrian': [3, 4, 6, 8, 9, 10, 12, 13, 14, 15, 16, 20, 21, 32, 35, 36, 38, 47, 49, 64, 74, 91, 122], 'Cyclist': [], 'Other': [], 'sdc_index': 5, 'objects_of_interest': array([0, 4]), 'tracks_to_predict': array([0, 1, 2, 3, 4])}
[0 4]
[0 1 2 3 4]


## Generate visualization images.

- 将车辆与其他类型的actor区分开绘制。
- 将交互车辆和主车区分。

In [10]:
def create_figure_and_axes(size_pixels):
    """Initializes a unique figure and axes for plotting."""
    fig, ax = plt.subplots(1, 1, num=uuid.uuid4())

    # Sets output image to pixel resolution.
    dpi = 100
    size_inches = size_pixels / dpi
    fig.set_size_inches([size_inches, size_inches])
    fig.set_dpi(dpi)
    fig.set_facecolor('white')
    ax.set_facecolor('white')
    ax.xaxis.label.set_color('black')
    ax.tick_params(axis='x', colors='black')
    ax.yaxis.label.set_color('black')
    ax.tick_params(axis='y', colors='black')
    fig.set_tight_layout(True)
    ax.grid(False)
    return fig, ax


def fig_canvas_image(fig):
    """Returns a [H, W, 3] uint8 np.array image from fig.canvas.tostring_rgb()."""
    # Just enough margin in the figure to display xticks and yticks.
    fig.subplots_adjust(
        left=0.08, bottom=0.08, right=0.98, top=0.98, wspace=0.0, hspace=0.0)
    fig.canvas.draw()
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    return data.reshape(fig.canvas.get_width_height()[::-1] + (3,))


def get_colormap(num_agents):
    """Compute a color map array of shape [num_agents, 4]."""
    colors = cm.get_cmap('jet', num_agents)
    colors = colors(range(num_agents))
    np.random.shuffle(colors)
    return colors


def get_viewport(all_states, all_states_mask):
    """Gets the region containing the data.

    Args:
      all_states: states of agents as an array of shape [num_agents, num_steps,
        2].
      all_states_mask: binary mask of shape [num_agents, num_steps] for
        `all_states`.

    Returns:
      center_y: float. y coordinate for center of data.
      center_x: float. x coordinate for center of data.
      width: float. Width of data.
    """
    valid_states = all_states[all_states_mask]
    all_y = valid_states[..., 1]
    all_x = valid_states[..., 0]

    center_y = (np.max(all_y) + np.min(all_y)) / 2
    center_x = (np.max(all_x) + np.min(all_x)) / 2

    range_y = np.ptp(all_y)
    range_x = np.ptp(all_x)

    width = max(range_y, range_x)

    return center_y, center_x, width


def visualize_one_step(states,
                       mask,
                       roadgraph,
                       title,
                       center_y,
                       center_x,
                       width,
                       color_map,
                       size_pixels=1000,
                       actor_indices=None,
                       decoded_example=None):
    """Generate visualization for a single step."""

    # Create figure and axes.
    fig, ax = create_figure_and_axes(size_pixels=size_pixels)

    # Plot roadgraph.
    rg_pts = roadgraph[:, :2].T
    ax.plot(rg_pts[0, :], rg_pts[1, :], 'k.', alpha=1, ms=2)

    # Define markers and colors based on actor type
    markers = ['o', 'o', 'o', 'o']  # Default marker is circle 'o'
    # print(f"markers: {markers[1]}")
    colors = [(0.5, 0.5, 0.5), (0, 0, 1), (1, 1, 0), (0, 1, 0), (0.6, 0.4, 0.2)]  # Gray, Blue, Yellow, Green, Brown

    masked_x = states[:, 0][mask]
    masked_y = states[:, 1][mask]
    types = decoded_example['state/type'][mask].numpy().astype(int)
    # print(f"types: {types}, type of types: {type(types)}")

    # Plot agent current position.
    # 绘制普通代理
    for idx, (x, y, t) in enumerate(zip(masked_x, masked_y, types)):
        # print(f"idx: {idx}, x: {x}, y: {y}, t: {t}")
        # print(f"markers[t - 1]:{markers[t - 1]}")
        ax.scatter(x, y,
                   marker=markers[t - 1],
                   s=50,  # 普通代理大小
                   linewidths=1.5,
                   color=colors[t],
                   alpha=0.7)
    # 如果存在主车索引，单独绘制
    # If there exists main car index, plot it separately
    if actor_indices and 'sdc_index' in actor_indices and actor_indices['sdc_index'] is not None and mask[
        actor_indices['sdc_index']]:
        sdc_x = states[actor_indices['sdc_index'], 0]
        sdc_y = states[actor_indices['sdc_index'], 1]
        ax.scatter([sdc_x], [sdc_y],
                   marker='*',  # Star marker
                   s=500,  # Large size
                   linewidths=1,
                   color='red',  # Red
                   edgecolors='black',  # Black border
                   zorder=3)  # Ensure it's on top

    # 如果存在有需要预测轨迹的车辆，单独绘制
    # If there exist interacting vehicles, plot them separately
    if actor_indices and 'tracks_to_predict' in actor_indices:
        for predict in actor_indices['tracks_to_predict']:
            if predict is not None and mask[predict]:
                predict_x = states[predict, 0]
                predict_y = states[predict, 1]
                ax.scatter([predict_x], [predict_y],
                           marker='o',  # Star marker
                           s=300,  # Large size
                           linewidths=1,
                           color='#d215e1',  # Indigo blue
                           # color=(0, 0, 0.5),  # Indigo blue
                           edgecolors='black',  # Black border
                           alpha=0.7,
                           zorder=3)  # Ensure it's on top

    # 如果存在有交互的车辆，单独绘制
    # If there exist interacting vehicles, plot them separately
    if actor_indices and 'objects_of_interest' in actor_indices:
        for interest in actor_indices['objects_of_interest']:
            if interest is not None and mask[interest]:
                interest_x = states[interest, 0]
                interest_y = states[interest, 1]
                ax.scatter([interest_x], [interest_y],
                           marker='*',  # Star marker
                           s=500,  # Large size
                           linewidths=1,
                           color=(0, 0, 0.5),  # Indigo blue
                           edgecolors='black',  # Black border
                           zorder=3)  # Ensure it's on top

    # Title.
    ax.set_title(title)

    # Set axes.  Should be at least 10m on a side and cover 160% of agents.
    size = max(10, width * 1.0)
    ax.axis([
        -size / 2 + center_x, size / 2 + center_x, -size / 2 + center_y,
        size / 2 + center_y
    ])
    ax.set_aspect('equal')

    image = fig_canvas_image(fig)
    plt.close(fig)
    return image


def visualize_all_agents_smooth(
        decoded_example,
        actor_indices=None,
        size_pixels=1000,
):
    """Visualizes all agent predicted trajectories in a serie of images.

    Args:
      decoded_example: Dictionary containing agent info about all modeled agents.
      size_pixels: The size in pixels of the output image.

    Returns:
      T of [H, W, 3] uint8 np.arrays of the drawn matplotlib's figure canvas.
    """
    # [num_agents, num_past_steps, 2] float32.
    past_states = tf.stack(
        [decoded_example['state/past/x'], decoded_example['state/past/y']],
        -1).numpy()
    past_states_mask = decoded_example['state/past/valid'].numpy() > 0.0

    # [num_agents, 1, 2] float32.
    current_states = tf.stack(
        [decoded_example['state/current/x'], decoded_example['state/current/y']],
        -1).numpy()
    current_states_mask = decoded_example['state/current/valid'].numpy() > 0.0

    # [num_agents, num_future_steps, 2] float32.
    future_states = tf.stack(
        [decoded_example['state/future/x'], decoded_example['state/future/y']],
        -1).numpy()
    future_states_mask = decoded_example['state/future/valid'].numpy() > 0.0

    # [num_points, 3] float32.
    roadgraph_xyz = decoded_example['roadgraph_samples/xyz'].numpy()

    num_agents, num_past_steps, _ = past_states.shape
    num_future_steps = future_states.shape[1]

    color_map = get_colormap(num_agents)

    # [num_agens, num_past_steps + 1 + num_future_steps, depth] float32.
    all_states = np.concatenate([past_states, current_states, future_states], 1)

    # [num_agens, num_past_steps + 1 + num_future_steps] float32.
    all_states_mask = np.concatenate(
        [past_states_mask, current_states_mask, future_states_mask], 1)

    center_y, center_x, width = get_viewport(all_states, all_states_mask)

    images = []

    # Generate images from past time steps.

    for i, (s, m) in enumerate(
            zip(np.split(past_states, num_past_steps, 1), np.split(past_states_mask, num_past_steps, 1))):
        # print(f"i: {i}, s: {s}, m: {m}")
        im = visualize_one_step(s[:, 0], m[:, 0], roadgraph_xyz,
                                'past: %d' % (num_past_steps - i), center_y,
                                center_x, width, color_map, size_pixels, actor_indices, decoded_example)
        images.append(im)

    # Generate one image for the current time step.
    s = current_states
    m = current_states_mask

    im = visualize_one_step(s[:, 0], m[:, 0], roadgraph_xyz, 'current', center_y,
                            center_x, width, color_map, size_pixels, actor_indices, decoded_example)
    images.append(im)

    # Generate images from future time steps.
    for i, (s, m) in enumerate(
            zip(
                np.split(future_states, num_future_steps, 1),
                np.split(future_states_mask, num_future_steps, 1))):
        im = visualize_one_step(s[:, 0], m[:, 0], roadgraph_xyz,
                                'future: %d' % (i + 1), center_y, center_x, width,
                                color_map, size_pixels, actor_indices, decoded_example)
        images.append(im)

    return images




In [11]:
# 获取actor索引
actor_indices = get_actor_indices(parsed)
# 可视化所有代理的平滑轨迹
images = visualize_all_agents_smooth(parsed, actor_indices=actor_indices)

sdc_index[5]
objects_of_interest: [0 4]
state/tracks_to_predict: [0 1 2 3 4]


/tmp/ipykernel_163831/3733787317.py:33: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = cm.get_cmap('jet', num_agents)


## Display animation.

In [12]:
def create_animation(images):
    """ Creates a Matplotlib animation of the given images.

    Args:
      images: A list of numpy arrays representing the images.

    Returns:
      A matplotlib.animation.Animation.

    Usage:
      anim = create_animation(images)
      anim.save('/tmp/animation.avi')
      HTML(anim.to_html5_video())
    """

    plt.ioff()
    fig, ax = plt.subplots()
    dpi = 100
    size_inches = 1000 / dpi
    fig.set_size_inches([size_inches, size_inches])
    plt.ion()

    def animate_func(i):
        ax.imshow(images[i])
        ax.set_xticks([])
        ax.set_yticks([])
        ax.grid('off')

    anim = animation.FuncAnimation(
        fig, animate_func, frames=len(images) // 2, interval=100)
    plt.close(fig)
    return anim


anim = create_animation(images[::1])

HTML(anim.to_html5_video())



保存成MP4方便观看

In [13]:
anim.save('./video/animation.mp4',
          writer=animation.FFMpegWriter(fps=10, bitrate=5000),
          dpi=100)

# 提取车辆动作数据

根据交互车辆的状态和动作去预测——提取车辆的位置和速度

- `state/*/x`：X轴位置
- `state/*/y`：Y轴位置
- `state/*/z`：Z轴位置
- `state/*/bbox_yaw`：车子的朝向
- `state/*/speed`：车子的速度（标量）
- `state/*/vel_yaw`：车子的速度（方向角）
- `state/*/velocity_x`：车子的速度（x方向）
- `state/*/velocity_y`：车子的速度（y方向）
- `state/*/timestamp_micros`：时间戳
- `state/*/valid`：是否有效，为零的话是无效

In [23]:
objects_of_interest = parsed['state/objects_of_interest']
to_predict = parsed['state/tracks_to_predict']

valid_past = parsed['state/past/valid']
valid_current = parsed['state/current/valid']
valid_future = parsed['state/future/valid']

print(f"valid_past: {valid_past}")
print(f"state/past/valid: {parsed['state/past/valid']}")

valid_past: [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
state/past/valid: [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


## 从原始数据读取

仅仅保留有用的部分

In [24]:
# If you use a custom conversion from Scenario to tf.Example, set the correct
# number of map samples here.
num_map_samples = 30000

# Features of other agents.
state_features = {
    'state/id':
        tf.io.FixedLenFeature([128], tf.float32, default_value=None),
    'state/type':
        tf.io.FixedLenFeature([128], tf.float32, default_value=None),
    'state/is_sdc':
        tf.io.FixedLenFeature([128], tf.int64, default_value=None),
    'state/tracks_to_predict':  #一个标志向量，用于指示要预测哪些对象对应于 state/XX 张量的第一个维度。最多可选择 8 个对象。对于训练集和验证集，这些指示提交 proto prediction 中必须存在哪个对象。如果在提交 proto 中没有准确预测这些对象，则提交将失败。此字段也存在于训练集中，作为要训练的对象的参考，但您可以自由选择要训练的其他对象。
        tf.io.FixedLenFeature([128], tf.int64, default_value=None),
    'state/objects_of_interest':
        tf.io.FixedLenFeature([128], tf.int64, default_value=None),

    'state/current/bbox_yaw':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/timestamp_micros':  # 每个时间步长的时间戳（以微秒为单位）。
        tf.io.FixedLenFeature([128, 1], tf.int64, default_value=None),
    'state/current/valid':  #功能 state/current/XX 的所有元素的有效标志。如果设置为 1，则元素将填充有效数据，否则将填充 -1。{0, 1}.指示 state/current/XX 张量的每个元素的数据是否填充了有效数据（而不是用 -1 填充）。
        tf.io.FixedLenFeature([128, 1], tf.int64, default_value=None),
    'state/current/vel_yaw':  #每个对象的速度向量在每个时间步长的偏转角。角度（以弧度为单位）。这是在 X-Y 平面中沿 X 方向旋转单位向量以与对象边界框的速度向量对齐的角度。
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/velocity_x':  #每个时间步长处对象速度的 x 分量。
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/velocity_y':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/x':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/y':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/z':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),
    'state/current/speed':
        tf.io.FixedLenFeature([128, 1], tf.float32, default_value=None),

    'state/future/bbox_yaw':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/timestamp_micros':
        tf.io.FixedLenFeature([128, 80], tf.int64, default_value=None),
    'state/future/valid':
        tf.io.FixedLenFeature([128, 80], tf.int64, default_value=None),
    'state/future/vel_yaw':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/velocity_x':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/velocity_y':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/x':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/y':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/z':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),
    'state/future/speed':
        tf.io.FixedLenFeature([128, 80], tf.float32, default_value=None),

    'state/past/bbox_yaw':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/timestamp_micros':
        tf.io.FixedLenFeature([128, 10], tf.int64, default_value=None),
    'state/past/valid':
        tf.io.FixedLenFeature([128, 10], tf.int64, default_value=None),
    'state/past/vel_yaw':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/velocity_x':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/velocity_y':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/x':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/y':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/z':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
    'state/past/speed':
        tf.io.FixedLenFeature([128, 10], tf.float32, default_value=None),
}

features_description_extract = {}
features_description_extract.update(state_features)
parsed_extract = tf.io.parse_single_example(data, features_description_extract)


def extract_tracks_to_predict(parsed_example):
    # 提取 state/tracks_to_predict 特征
    tracks_to_predict = parsed_example['state/tracks_to_predict']

    # 创建一个掩码，其中 tracks_to_predict 为真（1）
    mask = tf.cast(tracks_to_predict, tf.bool)

    # 使用掩码提取所有相关数据
    extracted_data = {}
    for key, value in parsed_example.items():
        if len(value.shape) == 1:
            extracted_data[key] = tf.boolean_mask(value, mask)
        elif len(value.shape) == 2:
            # 使用掩码提取二维张量中的行
            extracted_data[key] = tf.boolean_mask(value, mask, axis=0)

    return extracted_data


# 提取 tracks_to_predict 为真的 actor 数据
extracted_actors = extract_tracks_to_predict(parsed_extract)# 可以点击这个变量查看里面的数据
print(f"extracted_actors: {extracted_actors}")

extracted_actors: {'state/current/bbox_yaw': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[-5.3723035],
       [-4.842374 ],
       [-5.698106 ],
       [-1.3935251],
       [-1.357977 ]], dtype=float32)>, 'state/current/speed': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[2.154818 ],
       [0.6746282],
       [3.6880183],
       [1.4093947],
       [1.5491388]], dtype=float32)>, 'state/current/timestamp_micros': <tf.Tensor: shape=(5, 1), dtype=int64, numpy=
array([[1000110],
       [1000110],
       [1000110],
       [1000110],
       [1000110]])>, 'state/current/valid': <tf.Tensor: shape=(5, 1), dtype=int64, numpy=
array([[1],
       [1],
       [1],
       [1],
       [1]])>, 'state/current/vel_yaw': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[ 0.9152993],
       [ 1.3741105],
       [ 0.6657787],
       [-1.4037254],
       [-1.3547876]], dtype=float32)>, 'state/current/velocity_x': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[1.3134766 

## 将提取的数据写入新的tf文件（还跑不了）

In [20]:
# 将提取的数据转换为 tf.Example 并写入 TFRecord 文件
def create_tf_example(extracted_data):
    feature_dict = {}
    for key, value in extracted_data.items():
        if value.dtype == tf.float32:
            feature_dict[key] = tf.train.Feature(float_list=tf.train.FloatList(value=value.numpy().flatten()))
        elif value.dtype == tf.int64:
            feature_dict[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=value.numpy().flatten()))

    return tf.train.Example(features=tf.train.Features(feature=feature_dict))


# 创建 tf.Example
tf_example = create_tf_example(extracted_actors)
# 写入 TFRecord 文件
output_file = "extracted_actors.tfrecord"
with tf.io.TFRecordWriter(output_file) as writer:
    writer.write(tf_example.SerializeToString())

print(f"Data saved to {output_file}")

{'state/current/bbox_yaw': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[-5.3723035],
       [-4.842374 ],
       [-5.698106 ],
       [-1.3935251],
       [-1.357977 ]], dtype=float32)>, 'state/current/speed': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[2.154818 ],
       [0.6746282],
       [3.6880183],
       [1.4093947],
       [1.5491388]], dtype=float32)>, 'state/current/timestamp_micros': <tf.Tensor: shape=(5, 1), dtype=int64, numpy=
array([[1000110],
       [1000110],
       [1000110],
       [1000110],
       [1000110]])>, 'state/current/valid': <tf.Tensor: shape=(5, 1), dtype=int64, numpy=
array([[1],
       [1],
       [1],
       [1],
       [1]])>, 'state/current/vel_yaw': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[ 0.9152993],
       [ 1.3741105],
       [ 0.6657787],
       [-1.4037254],
       [-1.3547876]], dtype=float32)>, 'state/current/velocity_x': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[1.3134766 ],
       [0.13183

In [50]:
# 读取 TFRecord 文件并输出数据
def parse_tf_example(proto):
    return tf.io.parse_single_example(proto, features_description_extract)


# 创建 Dataset 从 TFRecord 文件中读取数据
parsed_extract = tf.io.parse_single_example(data, features_description_extract)

# 提取并打印数据
for parsed_record in dataset:
    print("\nParsed Record from TFRecord:")
    for key, value in parsed_record.items():
        print(f"{key}: {value.numpy()}")

2025-04-07 11:09:44.504191: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2025-04-07 11:09:44.519994: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: state/tracks_to_predict.  Can't parse serialized Example.
2025-04-07 11:09:44.520030: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: Key: state/tracks_to_predict.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_35_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: state/tracks_to_predict.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext]

# Simple MLP model with TF

Note that this is a very simple example model to demonstrate inputs parsing and metrics computation. Not at all competitive.

In [30]:
def _parse(value):
    decoded_example = tf.io.parse_single_example(value, features_description)

    past_states = tf.stack([
        decoded_example['state/past/x'], decoded_example['state/past/y'],
        decoded_example['state/past/length'], decoded_example['state/past/width'],
        decoded_example['state/past/bbox_yaw'],
        decoded_example['state/past/velocity_x'],
        decoded_example['state/past/velocity_y']
    ], -1)

    cur_states = tf.stack([
        decoded_example['state/current/x'], decoded_example['state/current/y'],
        decoded_example['state/current/length'],
        decoded_example['state/current/width'],
        decoded_example['state/current/bbox_yaw'],
        decoded_example['state/current/velocity_x'],
        decoded_example['state/current/velocity_y']
    ], -1)

    input_states = tf.concat([past_states, cur_states], 1)[..., :2]

    future_states = tf.stack([
        decoded_example['state/future/x'], decoded_example['state/future/y'],
        decoded_example['state/future/length'],
        decoded_example['state/future/width'],
        decoded_example['state/future/bbox_yaw'],
        decoded_example['state/future/velocity_x'],
        decoded_example['state/future/velocity_y']
    ], -1)

    gt_future_states = tf.concat([past_states, cur_states, future_states], 1)

    past_is_valid = decoded_example['state/past/valid'] > 0
    current_is_valid = decoded_example['state/current/valid'] > 0
    future_is_valid = decoded_example['state/future/valid'] > 0
    gt_future_is_valid = tf.concat(
        [past_is_valid, current_is_valid, future_is_valid], 1)

    # If a sample was not seen at all in the past, we declare the sample as
    # invalid.
    sample_is_valid = tf.reduce_any(
        tf.concat([past_is_valid, current_is_valid], 1), 1)

    inputs = {
        'input_states': input_states,
        'gt_future_states': gt_future_states,
        'gt_future_is_valid': gt_future_is_valid,
        'object_type': decoded_example['state/type'],
        'tracks_to_predict': decoded_example['state/tracks_to_predict'] > 0,
        'sample_is_valid': sample_is_valid,
    }
    return inputs


def _default_metrics_config():
    config = motion_metrics_pb2.MotionMetricsConfig()
    config_text = """
  track_steps_per_second: 10
  prediction_steps_per_second: 2
  track_history_samples: 10
  track_future_samples: 80
  speed_lower_bound: 1.4
  speed_upper_bound: 11.0
  speed_scale_lower: 0.5
  speed_scale_upper: 1.0
  step_configurations {
    measurement_step: 5
    lateral_miss_threshold: 1.0
    longitudinal_miss_threshold: 2.0
  }
  step_configurations {
    measurement_step: 9
    lateral_miss_threshold: 1.8
    longitudinal_miss_threshold: 3.6
  }
  step_configurations {
    measurement_step: 15
    lateral_miss_threshold: 3.0
    longitudinal_miss_threshold: 6.0
  }
  max_predictions: 6
  """
    text_format.Parse(config_text, config)
    return config


class SimpleModel(tf.keras.Model):
    """A simple one-layer regressor."""

    def __init__(self, num_agents_per_scenario, num_states_steps,
                 num_future_steps):
        super(SimpleModel, self).__init__()
        self._num_agents_per_scenario = num_agents_per_scenario
        self._num_states_steps = num_states_steps
        self._num_future_steps = num_future_steps
        self.regressor = tf.keras.layers.Dense(num_future_steps * 2)

    def call(self, states):
        states = tf.reshape(states, (-1, self._num_states_steps * 2))
        pred = self.regressor(states)
        pred = tf.reshape(
            pred, [-1, self._num_agents_per_scenario, self._num_future_steps, 2])
        return pred


class MotionMetrics(tf.keras.metrics.Metric):
    """Wrapper for motion metrics computation."""

    def __init__(self, config):
        super().__init__()
        self._prediction_trajectory = []
        self._prediction_score = []
        self._ground_truth_trajectory = []
        self._ground_truth_is_valid = []
        self._prediction_ground_truth_indices = []
        self._prediction_ground_truth_indices_mask = []
        self._object_type = []
        self._metrics_config = config

    def reset_state():
        self._prediction_trajectory = []
        self._prediction_score = []
        self._ground_truth_trajectory = []
        self._ground_truth_is_valid = []
        self._prediction_ground_truth_indices = []
        self._prediction_ground_truth_indices_mask = []
        self._object_type = []

    def update_state(self, prediction_trajectory, prediction_score,
                     ground_truth_trajectory, ground_truth_is_valid,
                     prediction_ground_truth_indices,
                     prediction_ground_truth_indices_mask, object_type):
        self._prediction_trajectory.append(prediction_trajectory)
        self._prediction_score.append(prediction_score)
        self._ground_truth_trajectory.append(ground_truth_trajectory)
        self._ground_truth_is_valid.append(ground_truth_is_valid)
        self._prediction_ground_truth_indices.append(
            prediction_ground_truth_indices)
        self._prediction_ground_truth_indices_mask.append(
            prediction_ground_truth_indices_mask)
        self._object_type.append(object_type)

    def result(self):
        # [batch_size, num_preds, 1, 1, steps, 2].
        # The ones indicate top_k = 1, num_agents_per_joint_prediction = 1.
        prediction_trajectory = tf.concat(self._prediction_trajectory, 0)
        # [batch_size, num_preds, 1].
        prediction_score = tf.concat(self._prediction_score, 0)
        # [batch_size, num_agents, gt_steps, 7].
        ground_truth_trajectory = tf.concat(self._ground_truth_trajectory, 0)
        # [batch_size, num_agents, gt_steps].
        ground_truth_is_valid = tf.concat(self._ground_truth_is_valid, 0)
        # [batch_size, num_preds, 1].
        prediction_ground_truth_indices = tf.concat(
            self._prediction_ground_truth_indices, 0)
        # [batch_size, num_preds, 1].
        prediction_ground_truth_indices_mask = tf.concat(
            self._prediction_ground_truth_indices_mask, 0)
        # [batch_size, num_agents].
        object_type = tf.cast(tf.concat(self._object_type, 0), tf.int64)

        # We are predicting more steps than needed by the eval code. Subsample.
        interval = (
                self._metrics_config.track_steps_per_second //
                self._metrics_config.prediction_steps_per_second)
        prediction_trajectory = prediction_trajectory[...,
                                (interval - 1)::interval, :]

        return py_metrics_ops.motion_metrics(
            config=self._metrics_config.SerializeToString(),
            prediction_trajectory=prediction_trajectory,
            prediction_score=prediction_score,
            ground_truth_trajectory=ground_truth_trajectory,
            ground_truth_is_valid=ground_truth_is_valid,
            prediction_ground_truth_indices=prediction_ground_truth_indices,
            prediction_ground_truth_indices_mask=prediction_ground_truth_indices_mask,
            object_type=object_type)


model = SimpleModel(128, 11, 80)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = tf.keras.losses.MeanSquaredError()
metrics_config = _default_metrics_config()
motion_metrics = MotionMetrics(metrics_config)
metric_names = config_util.get_breakdown_names_from_motion_config(
    metrics_config)


def train_step(inputs):
    with tf.GradientTape() as tape:
        # [batch_size, num_agents, D]
        states = inputs['input_states']

        # Predict. [batch_size, num_agents, steps, 2].
        pred_trajectory = model(states, training=True)

        # Set training target.
        prediction_start = metrics_config.track_history_samples + 1

        # [batch_size, num_agents, steps, 7]
        gt_trajectory = inputs['gt_future_states']
        gt_targets = gt_trajectory[..., prediction_start:, :2]

        # [batch_size, num_agents, steps]
        gt_is_valid = inputs['gt_future_is_valid']
        # [batch_size, num_agents, steps]
        weights = (
                tf.cast(inputs['gt_future_is_valid'][..., prediction_start:],
                        tf.float32) *
                tf.cast(inputs['tracks_to_predict'][..., tf.newaxis], tf.float32))

        loss_value = loss_fn(gt_targets, pred_trajectory, sample_weight=weights)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    # [batch_size, num_agents, steps, 2] ->
    # [batch_size, num_agents, 1, 1, steps, 2].
    # The added dimensions are top_k = 1, num_agents_per_joint_prediction = 1.
    pred_trajectory = pred_trajectory[:, :, tf.newaxis, tf.newaxis]

    # Fake the score since this model does not generate any score per predicted
    # trajectory.
    pred_score = tf.ones(shape=tf.shape(pred_trajectory)[:3])

    # [batch_size, num_agents].
    object_type = inputs['object_type']

    # [batch_size, num_agents].
    batch_size = tf.shape(inputs['tracks_to_predict'])[0]
    num_samples = tf.shape(inputs['tracks_to_predict'])[1]

    pred_gt_indices = tf.range(num_samples, dtype=tf.int64)
    # [batch_size, num_agents, 1].
    pred_gt_indices = tf.tile(pred_gt_indices[tf.newaxis, :, tf.newaxis],
                              (batch_size, 1, 1))
    # [batch_size, num_agents, 1].
    pred_gt_indices_mask = inputs['tracks_to_predict'][..., tf.newaxis]

    motion_metrics.update_state(pred_trajectory, pred_score, gt_trajectory,
                                gt_is_valid, pred_gt_indices,
                                pred_gt_indices_mask, object_type)

    return loss_value


dataset = tf.data.TFRecordDataset(FILENAME)
dataset = dataset.map(_parse)
dataset = dataset.batch(32)

epochs = 2
num_batches_per_epoch = 10

for epoch in range(epochs):
    print('\nStart of epoch %d' % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, batch in enumerate(dataset):
        loss_value = train_step(batch)

        # Log every 10 batches.
        if step % 10 == 0:
            print('Training loss (for one batch) at step %d: %.4f' %
                  (step, float(loss_value)))
            print('Seen so far: %d samples' % ((step + 1) * 64))

        if step >= num_batches_per_epoch:
            break

    # Display metrics at the end of each epoch.
    train_metric_values = motion_metrics.result()
    for i, m in enumerate(
            ['min_ade', 'min_fde', 'miss_rate', 'overlap_rate', 'map']):
        for j, n in enumerate(metric_names):
            print('{}/{}: {}'.format(m, n, train_metric_values[i, j]))


Start of epoch 0
Training loss (for one batch) at step 0: 1136246.2500
Seen so far: 64 samples


2025-04-02 16:43:58.875252: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


Training loss (for one batch) at step 10: 1086275.2500
Seen so far: 704 samples


2025-04-02 16:43:59.230470: I waymo_open_dataset/metrics/ops/motion_metrics_ops.cc:163] Computing motion metrics for 352 trajectories.
2025-04-02 16:43:59.230485: I waymo_open_dataset/metrics/ops/motion_metrics_ops.cc:166] Parsing prediction [352,128,1,1,16,2][352]
2025-04-02 16:43:59.230487: I waymo_open_dataset/metrics/ops/motion_metrics_ops.cc:170] Parsing ground truth [352,128,91,7][352]


min_ade/TYPE_VEHICLE_5: 7123.4375
min_ade/TYPE_VEHICLE_9: 6725.51220703125
min_ade/TYPE_VEHICLE_15: 6519.36474609375
min_ade/TYPE_PEDESTRIAN_5: 6543.85009765625
min_ade/TYPE_PEDESTRIAN_9: 6164.64111328125
min_ade/TYPE_PEDESTRIAN_15: 6084.38818359375
min_ade/TYPE_CYCLIST_5: 6483.07763671875
min_ade/TYPE_CYCLIST_9: 6077.8681640625
min_ade/TYPE_CYCLIST_15: 6036.91650390625
min_fde/TYPE_VEHICLE_5: 8301.400390625
min_fde/TYPE_VEHICLE_9: 4216.564453125
min_fde/TYPE_VEHICLE_15: 8046.22509765625
min_fde/TYPE_PEDESTRIAN_5: 7154.17041015625
min_fde/TYPE_PEDESTRIAN_9: 3798.66064453125
min_fde/TYPE_PEDESTRIAN_15: 7715.14501953125
min_fde/TYPE_CYCLIST_5: 6873.97705078125
min_fde/TYPE_CYCLIST_9: 3732.183837890625
min_fde/TYPE_CYCLIST_15: 7395.7392578125
miss_rate/TYPE_VEHICLE_5: 1.0
miss_rate/TYPE_VEHICLE_9: 1.0
miss_rate/TYPE_VEHICLE_15: 1.0
miss_rate/TYPE_PEDESTRIAN_5: 1.0
miss_rate/TYPE_PEDESTRIAN_9: 1.0
miss_rate/TYPE_PEDESTRIAN_15: 1.0
miss_rate/TYPE_CYCLIST_5: 1.0
miss_rate/TYPE_CYCLIST_9: 1.0

2025-04-02 16:43:59.800934: I waymo_open_dataset/metrics/ops/motion_metrics_ops.cc:163] Computing motion metrics for 704 trajectories.
2025-04-02 16:43:59.800958: I waymo_open_dataset/metrics/ops/motion_metrics_ops.cc:166] Parsing prediction [704,128,1,1,16,2][704]
2025-04-02 16:43:59.800960: I waymo_open_dataset/metrics/ops/motion_metrics_ops.cc:170] Parsing ground truth [704,128,91,7][704]


min_ade/TYPE_VEHICLE_5: 6701.23193359375
min_ade/TYPE_VEHICLE_9: 6296.15234375
min_ade/TYPE_VEHICLE_15: 6108.466796875
min_ade/TYPE_PEDESTRIAN_5: 6186.53955078125
min_ade/TYPE_PEDESTRIAN_9: 5806.89794921875
min_ade/TYPE_PEDESTRIAN_15: 5729.5048828125
min_ade/TYPE_CYCLIST_5: 6149.83837890625
min_ade/TYPE_CYCLIST_9: 5748.40576171875
min_ade/TYPE_CYCLIST_15: 5706.15625
min_fde/TYPE_VEHICLE_5: 7843.1142578125
min_fde/TYPE_VEHICLE_9: 3745.79052734375
min_fde/TYPE_VEHICLE_15: 7606.24072265625
min_fde/TYPE_PEDESTRIAN_5: 6784.01318359375
min_fde/TYPE_PEDESTRIAN_9: 3420.789306640625
min_fde/TYPE_PEDESTRIAN_15: 7333.03955078125
min_fde/TYPE_CYCLIST_5: 6539.12353515625
min_fde/TYPE_CYCLIST_9: 3383.37890625
min_fde/TYPE_CYCLIST_15: 7046.1484375
miss_rate/TYPE_VEHICLE_5: 1.0
miss_rate/TYPE_VEHICLE_9: 1.0
miss_rate/TYPE_VEHICLE_15: 1.0
miss_rate/TYPE_PEDESTRIAN_5: 1.0
miss_rate/TYPE_PEDESTRIAN_9: 1.0
miss_rate/TYPE_PEDESTRIAN_15: 1.0
miss_rate/TYPE_CYCLIST_5: 1.0
miss_rate/TYPE_CYCLIST_9: 1.0
miss_r